# Building an evaluation dataset for SQL Agent

This notebook is used for building an evaluation dataset containing at least the following:
- `question`
- `SQL query`
- `Natural Language answer parsed from query result`

Evaluation dataset is based on databases obtained from **Spider dataset**.

#### Necessary imports

In [1]:
import os
import pandas as pd
from getpass import getpass

from langchain import LLMChain
from langchain.callbacks import get_openai_callback
from langchain.chat_models import ChatOpenAI
from langchain.utilities import SQLDatabase

from utils.evaluation_prompts import TARGET_PROMPT
from utils.utils import CustomDatabase

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key: ")

/Users/manuelsoria/miniconda3/lib/python3.10/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.25) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


### Loading the whole evaluation dataset

You need **Spider dataset** inside `datasets/spider` for this part. You can download it [here](https://drive.google.com/uc?export=download&id=1TqleXec_OykOYFREKKtschzY29dUcVAQ)

In [2]:
databases_info = pd.read_json('datasets/spider/tables.json')
evaluation_df = pd.read_json('datasets/spider/train_spider.json')

In [4]:
# # Filter out databases with percentiles 25, 50, and 75 in terms of number of tables
# percentiles = [0.25, 0.5, 0.75]
# table_counts = databases_info['table_names'].apply(len)
# selected_databases_info = databases_info[table_counts.isin(table_counts.quantile(percentiles))]

# Filter only databases containing between 5 and 8 tables
filtered_databases_info = databases_info[
    (databases_info['table_names'].apply(len) >= 5) &
    (databases_info['table_names'].apply(len) <= 8)
].copy()

len(filtered_databases_info)



37

Based on these 37 interesting databases (huge filter out of over 150), let's now filter based on criteria on evaluation dataset:

In [5]:
evaluation_databases = []
for db_name in list(filtered_databases_info['db_id']):
    evaluation_dataset = evaluation_df[evaluation_df['db_id'] == db_name]
    database = SQLDatabase.from_uri(f'sqlite:///datasets/spider/database/{db_name}/{db_name}.sqlite')
    evaluation_database = CustomDatabase(name=db_name, database=database, evaluation_dataset=evaluation_dataset)
    evaluation_databases.append(evaluation_database)

insurance_fnol 42
store_product 44
dog_kennels 0
e_learning 82
products_for_hire 18
dorm_1 100
driving_school 93
music_2 100
sports_competition 52
csu_1 70
tracking_orders 60
insurance_policies 48
customers_campaigns_ecommerce 15
tracking_share_transactions 41
apartment_rentals 80
cre_Docs_and_Epenses 84
tracking_software_problems 48
products_gen_characteristics 86
riding_club 17
aircraft 46
restaurant_1 22
insurance_and_eClaims 40
college_1 164
local_govt_mdm 14
hr_1 124
soccer_1 14
real_estate_properties 0
college_3 74
yelp 0
car_1 0
geo 0
cre_Doc_Tracking_DB 90
activity_1 88
customers_and_addresses 88
customers_and_products_contacts 15
company_1 7
product_catalog 42


/Users/manuelsoria/miniconda3/lib/python3.10/site-packages/langchain/utilities/sql_database.py:111: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['11']; using no arguments.
  self._metadata.reflect(


We have a list of databases between 5 and 8 tables, and evaluation sets for each one of them.

Let's pick 3 databases containing ~20 evaluation questions to check how they look:

In [6]:
for e in evaluation_databases:
    if 15 < len(e.evaluation_dataset) < 25:
        print(e.name, len(e.evaluation_dataset))

products_for_hire 18
riding_club 17
restaurant_1 22


In [7]:
selected_databases_names = ['products_for_hire', 'riding_club', 'restaurant_1']

Filter out only interesting databases.

In [8]:
selected_databases = []
for db_name in selected_databases_names:
    evaluation_dataset = evaluation_df[evaluation_df['db_id'] == db_name]
    database = SQLDatabase.from_uri(f'sqlite:///datasets/spider/database/{db_name}/{db_name}.sqlite')
    evaluation_database = CustomDatabase(name=db_name, database=database, evaluation_dataset=evaluation_dataset)
    selected_databases.append(evaluation_database)

For **products_for_hire** database:

In [10]:
# selected_databases[0].evaluation_dataset[['question','query']]

For **riding_club** database:

In [63]:
# selected_databases[1].evaluation_dataset[['question','query']]

For **restaurant_1** database:

In [62]:
# selected_databases[2].evaluation_dataset[['question','query']]

Now we need to query the results of the `queries` inside the `evaluation dataset` for each `evaluation database`:

In [11]:
for db in selected_databases:
    db.run_queries()

18it [00:00, 3644.58it/s]
/Users/manuelsoria/Pampa/evaluate-sql-agent/utils/utils.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.evaluation_dataset['query_result'] = results
17it [00:00, 4502.03it/s]
/Users/manuelsoria/Pampa/evaluate-sql-agent/utils/utils.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.evaluation_dataset['query_result'] = results
22it [00:00, 4653.52it/s]
/Users/manuelsoria/Pampa/evaluate-sql-agent/utils/utils.py:17: SettingWithCopyWarning: 
A value is trying to be set 

And we parse these query results into a natural language output.

**Watch out for token usage in this part as it's using an LLM for parsing the query result into Natural Language**

In [12]:
llm = ChatOpenAI(temperature=0)
target_llm_chain = LLMChain(llm=llm,prompt=TARGET_PROMPT)

In [13]:
for db in selected_databases:
    with get_openai_callback() as cb:
        print(f"Parsing results for {db.name}...")
        db.parse_query_results(target_llm_chain)
        print(f"Used {cb.total_tokens} tokens")

Parsing results for products_for_hire...


Parsing results for products_for_hire: 100%|██████████| 18/18 [00:43<00:00,  2.41s/it]
/Users/manuelsoria/Pampa/evaluate-sql-agent/utils/utils.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.evaluation_dataset['nl_result'] = results


Used 3084 tokens
Parsing results for riding_club...


Parsing results for riding_club: 100%|██████████| 17/17 [00:40<00:00,  2.39s/it]
/Users/manuelsoria/Pampa/evaluate-sql-agent/utils/utils.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.evaluation_dataset['nl_result'] = results


Used 2971 tokens
Parsing results for restaurant_1...


Parsing results for restaurant_1: 100%|██████████| 22/22 [00:28<00:00,  1.29s/it]

Used 2543 tokens



/Users/manuelsoria/Pampa/evaluate-sql-agent/utils/utils.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.evaluation_dataset['nl_result'] = results


### Saving evaluation dataset

Now let's keep only relevant columns:

In [14]:
for db in selected_databases:
    db.evaluation_dataset[['db_id', 'question', 'query', 'query_result', 'nl_result']].to_json(f"datasets/pampa_dataset/{db.name}_eval_dataset.json")

We also need to manually move the `{db_name}.sqlite` files for these databases into `pampa_dataset` folder